In [1]:
from capo.analysis.utils import (
    get_results,
    aggregate_results,
    get_prompt_scores,
    generate_comparison_table,
)
from capo.analysis.visualizations import (
    plot_population_scores,
    plot_population_members,
    plot_population_scores_comparison,
    plot_length_score,
)

In [2]:
OPTIMS = ["CAPO", "EvoPromptGA", "OPRO", "PromptWizard"]
DATASETS = ["sst-5", "agnews", "copa", "gsm8k", "subj"]
MODELS = ["llama", "mistral", "qwen"]

In [3]:
%load_ext autoreload
%autoreload 2

# Population Comparison

In [ ]:
for dataset in DATASETS:
    for model in MODELS:
        plot_population_scores_comparison(
            dataset,
            model,
            OPTIMS,
            agg="mean",
            plot_seeds=True,
            plot_stddev=True,
            x_col="input_tokens_cum",
        )

# Table Results

In [9]:
# import display html
from IPython.display import display, HTML

In [12]:
for model in MODELS:
    print(f"Model: {model}")
    display(generate_comparison_table(model=model, cutoff_tokens=1_000_000))

Model: llama
Failed to load sst-5 for CAPO: No objects to concatenate
Failed to load agnews for CAPO: No objects to concatenate
Failed to load copa for CAPO: No objects to concatenate
Failed to load sst-5 for EvoPromptGA: No objects to concatenate
Failed to load copa for EvoPromptGA: No objects to concatenate
Failed to load gsm8k for EvoPromptGA: No objects to concatenate
Failed to load subj for EvoPromptGA: No objects to concatenate


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,nan±nan,nan±nan,nan±nan,nan±nan,90.2±nan,90.2
EvoPromptGA,88.7±0.5,nan±nan,nan±nan,nan±nan,nan±nan,88.73
OPRO,88.2±0.6,98.1±0.7,51.7±2.3,59.3±1.5,71.3±3.4,73.73
PromptWizard,23.3±0.2,50.3±0.4,39.3±18.5,32.8±2.1,51.9±0.3,39.55


Model: mistral


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,84.1±1.9,96.6±nan,nan±nan,56.7±1.4,77.0±2.8,78.62
EvoPromptGA,83.9±1.5,96.3±0.8,41.1±3.6,54.2±2.4,77.8±0.0,70.67
OPRO,84.2±0.2,96.3±0.4,43.5±0.6,54.9±0.5,76.9±0.4,71.17
PromptWizard,44.4±31.6,57.5±12.6,48.7±7.9,31.1±4.7,59.0±6.2,48.12


Model: qwen
Failed to load copa for EvoPromptGA: No objects to concatenate
Failed to load gsm8k for EvoPromptGA: No objects to concatenate


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,87.3±1.0,98.8±0.3,54.4±nan,59.9±1.1,87.1±2.2,77.49
EvoPromptGA,80.3±0.9,nan±nan,nan±nan,57.1±0.8,71.9±1.9,69.76
OPRO,80.0±0.0,98.3±0.1,46.3±3.8,57.4±0.2,69.4±0.7,70.29
PromptWizard,63.5±34.9,98.1±0.2,15.3±24.7,39.7±15.1,64.9±6.1,56.31


In [13]:
for model in MODELS:
    print(f"Model: {model}")
    display(generate_comparison_table(model=model, cutoff_tokens=3_000_000))

Model: llama
Failed to load sst-5 for CAPO: No objects to concatenate
Failed to load agnews for CAPO: No objects to concatenate
Failed to load copa for CAPO: No objects to concatenate
Failed to load sst-5 for EvoPromptGA: No objects to concatenate
Failed to load copa for EvoPromptGA: No objects to concatenate
Failed to load gsm8k for EvoPromptGA: No objects to concatenate
Failed to load subj for EvoPromptGA: No objects to concatenate


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,nan±nan,nan±nan,71.4±nan,nan±nan,85.4±nan,78.4
EvoPromptGA,88.9±0.6,nan±nan,nan±nan,nan±nan,nan±nan,88.93
OPRO,88.2±0.6,98.1±0.7,51.9±2.5,61.7±1.3,71.3±3.4,74.24
PromptWizard,23.3±0.2,50.3±0.4,39.3±18.5,32.8±2.1,51.9±0.3,39.55


Model: mistral


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,84.0±2.2,94.6±nan,63.2±4.7,60.2±0.4,80.1±1.2,76.41
EvoPromptGA,84.4±0.3,96.1±0.4,43.0±3.5,54.6±1.0,73.7±4.0,70.37
OPRO,84.2±0.2,96.3±0.4,43.5±0.6,54.9±0.5,76.9±0.4,71.17
PromptWizard,44.4±31.6,57.5±12.6,48.7±7.9,31.1±4.7,59.0±6.2,48.12


Model: qwen
Failed to load copa for EvoPromptGA: No objects to concatenate
Failed to load gsm8k for EvoPromptGA: No objects to concatenate


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,87.0±0.7,98.3±0.4,58.5±3.6,59.9±0.9,89.9±1.8,78.74
EvoPromptGA,81.7±1.2,nan±nan,nan±nan,58.1±2.9,75.2±3.9,71.64
OPRO,79.9±0.2,98.3±0.1,46.3±3.8,57.4±0.2,70.7±2.9,70.51
PromptWizard,63.5±34.9,98.1±0.2,15.3±24.7,39.7±15.1,64.9±6.1,56.31


In [11]:
for model in MODELS:
    print(f"Model: {model}")
    display(generate_comparison_table(model=model))

Model: llama
Failed to load sst-5 for CAPO: No objects to concatenate
Failed to load agnews for CAPO: No objects to concatenate
Failed to load copa for CAPO: No objects to concatenate
Failed to load sst-5 for EvoPromptGA: No objects to concatenate
Failed to load copa for EvoPromptGA: No objects to concatenate
Failed to load gsm8k for EvoPromptGA: No objects to concatenate
Failed to load subj for EvoPromptGA: No objects to concatenate


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,nan±nan,nan±nan,71.4±nan,nan±nan,94.6±nan,83.0
EvoPromptGA,88.7±0.5,nan±nan,nan±nan,nan±nan,nan±nan,88.67
OPRO,88.2±0.6,98.1±0.7,51.9±2.5,60.9±1.3,71.3±3.4,74.07
PromptWizard,23.3±0.2,50.3±0.4,39.3±18.5,32.8±2.1,51.9±0.3,39.55


Model: mistral


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,84.3±2.6,94.6±nan,65.1±1.5,60.2±0.4,81.7±2.0,77.17
EvoPromptGA,84.4±0.3,96.1±0.4,43.9±4.7,54.9±1.2,74.9±2.5,70.87
OPRO,84.2±0.2,96.3±0.4,43.5±0.6,54.9±0.5,76.9±0.4,71.17
PromptWizard,44.4±31.6,57.5±12.6,48.7±7.9,31.1±4.7,59.0±6.2,48.12


Model: qwen
Failed to load copa for EvoPromptGA: No objects to concatenate
Failed to load gsm8k for EvoPromptGA: No objects to concatenate


,agnews,copa,gsm8k,sst-5,subj,avg
CAPO,87.1±1.0,98.4±0.3,60.2±5.9,59.1±0.6,91.0±0.8,79.15
EvoPromptGA,81.7±2.1,nan±nan,nan±nan,58.6±2.1,75.9±4.4,72.07
OPRO,79.9±0.2,98.5±0.5,46.3±3.8,57.0±0.5,70.7±2.9,70.47
PromptWizard,63.5±34.9,98.1±0.2,15.3±24.7,39.7±15.1,64.9±6.1,56.31


In [ ]:
for dataset in DATASETS:
    for model in MODELS:
        plot_length_score(
            dataset,
            model,
            OPTIMS,
            x_col="prompt_len",
            score_col="test_score",
        )